# Comparison of supervised machine learning classifiers for spam detection

<br/>

The performance of the three ML models for classifying email text data as spam are compared:

1. **Support Vector Machine (SVM)**

1. **Naive Bayes - Multinomial and Complement**

1. **Random Forest**


These models were selected as they are good for classifying text data.


Spam email data was split into training and testing subsets in 0.75 to 0. 25 ratio. 


For each model the split, train and test cycle are repeated 100 times, logging the accuracy, and reporting its mean and standard deviation (S.D.).

#### **Libraries**

First, importing the libraries.

In [ ]:
import os 
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import time
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import wordcloud

# ML learning model selection, metrics, feature extraction
from sklearn import ensemble, feature_extraction, model_selection, naive_bayes, metrics, svm

# To see available files
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

#### **Getting and cleaning data**

Next, reading in data, looking at the variables, and removing unwanted columns.

In [ ]:
# Read into dataframe
file_path = '../input/spam-email-data/spam.csv'
data = pd.read_csv(file_path)

# Drop extra columns; use data.columns to see column names
#data = data.drop(data.columns[2:], axis=1)
data.head(n=8) # Look at data, equivalent to data.iloc[:n]
# print(data.shape)


#### **Exploring data**

Next, checking the proportion of spam and non-spam emails in the dataset. 

In [ ]:
# Calculate spam ratio, to understand proportion of dataset that is spam:
spam_count = sum(True for entry in data.v1 if entry == 'spam')
ham_count = sum(True for entry in data.v1 if entry == 'ham')
spam_f = spam_count/len(data.v1)
ham_f = ham_count/len(data.v1)

print('Total count is {} \n'.format(spam_count+ham_count))
print('Spam fraction = {} / {} = {:.3f} \n\nHam fraction = {} / {} = {:.3f} \n\n'.format(spam_count,len(data.v1),spam_f, ham_count,len(data.v1),ham_f))

# Plot pie chart of spam and ham
colors = [(0.1, 0.9, 0.9, 0.7),(0.9, 0.2, 0.2, 0.7)]
plt.pie([ham_count,spam_count],  labels = ['non-spam: {:.3f}'.format(ham_f),'spam: {:.3f}'.format(spam_f)], colors = colors)
plt.title("Proportion of email that is spam")
plt.show

# Examine content of columns

# Check for Na/NaN 

# Check for duplicates

### Generate word clouds 
See common words in spam / non-spam emails

In [ ]:
# Select only spam emails and make one string with all spam contents
spam_only = data.v2[data.v1 == 'spam']
spam_text = " ".join(email for email in spam_only)

# similarly for non-spam
ham_only = data.v2[data.v1 == 'ham']
ham_text = " ".join(email for email in ham_only)

# spam_text[:1000]
# ham_text[:1000]

# Create word clouds for spam and ham texts using the .generate() method from wordcloud.WordCloud
spam_cloud = wordcloud.WordCloud(collocations = False, background_color = 'white',
                        width = 2048, height = 1080).generate(spam_text)
ham_cloud = wordcloud.WordCloud(collocations = False, background_color = 'white',
                        width = 2048, height = 1080).generate(ham_text)
# saving the image
spam_cloud.to_file('got_spam.png')
ham_cloud.to_file('got_ham.png')

In [ ]:
# figure size in inches optional
mpl.rcParams['figure.figsize'] = 18,6

# display two word clouds side by side 
fig, ax = plt.subplots(1,2, constrained_layout=False, sharey=True)

ax[0].set_title('Spam \n',fontsize=16)
ax[0].imshow(spam_cloud)
ax[0].axis('off') # turn off axes
# ax[0].set_xlabel('Spam \n',fontsize=16)
ax[1].set_title('Non-spam \n', fontsize=16)
ax[1].imshow(ham_cloud)
ax[1].axis('off')
fig.suptitle('Common words in spam and non-spam emails', fontsize= 20)

### Set the prediction features and target

* Prediction feature - email content -  independent variable

* Prediction target - *spam* or *ham* label - dependent variable

In [ ]:
# Set prediction target and feature variable
X = data.v2 # Email text is the feature used in prediction
y = data.v1 # Spam/ ham classification - prediciton target

## Support Vector Machine (SVM)

The SVM classification algorithm is applied as outlined in  [this tutorial](https://blog.logrocket.com/email-spam-detector-python-machine-learning/), using the same data.

First, the data is split into training (fitting) and testing subsets, the SVM model is created and trained, and finally tested in its ability to correctly identify spam in unseen data. 

This procedure is repeated and logged to calculate mean and S.D. of prediction accuaracy.

In [ ]:
# Record time to measure model runtime
start_time = time.time()
# List to log accuracy of N runs
accuracy_log = []
N = 40
for i in range(N):
    # Split data into train and test sets
    X_train, X_test,y_train, y_test = model_selection.train_test_split(X,y,test_size = 0.25)

    # Feature extraction
    cv = feature_extraction.text.CountVectorizer()
    features = cv.fit_transform(X_train)

    # Create and train SVM model
    model = svm.SVC()
    model.fit(features,y_train)
    #print('SVM Model fitted')

    features_test = cv.transform(X_test)
    accuracy_log.append(model.score(features_test,y_test))

# Log SVM model runtime
execution_time_svm = (time.time() - start_time)


# Calculate meand and SD of prediction accuracy for SVM model
svm_mean = np.mean(accuracy_log)
svm_sd = np.std(accuracy_log)

print( '\n')
print(accuracy_log)
print( '\n')
print("Data split for SVM train and test {} times.".format(N))
print( '\n')
print('Execution time in seconds: ' + str(round(execution_time_svm,1)))
print( '\n')
print("Accuracy mean and S.D  are {:.3f} and {:.3f} ".format(svm_mean, svm_sd))

#### Questions on SVM:

* Can I see the **support vectors** to help with **model explainability, contextualisation and visualisation**

* Margins - hard/ soft - how to define

## Naive Bayes

The **Naive Bayes** classifier is a **simple probabilistic ML classification model**. Based on statistical classfication using the Bayes Theorem, it assumes no dependence between features used in prediction, and works best when this is true.




### Mutlinomial Naive Bayes (MNB) 

The MNB is often used to classify and categorise text according to word frequency, and is suited to classifying email as spam.

In [ ]:
start_time = time.time() # Record time to measure model runtime

mnb_log = [] # log accuracy of each split
N = 40 # number splits
for i in range(N):
    # Split data into train and test sets
    X_train, X_test,y_train, y_test = model_selection.train_test_split(X,y,test_size = 0.25)

    # Feature extraction
    cv = feature_extraction.text.CountVectorizer()
    features = cv.fit_transform(X_train)

    # Fit MNB model
    model_mnb = naive_bayes.MultinomialNB()
    model_mnb.fit(features, y_train)
    
    # Test model
    features_test = cv.transform(X_test)
    mnb_log.append(model_mnb.score(features_test,y_test))

# Log MNB model runtime
execution_time_mnb = (time.time() - start_time)


mnb_mean = np.mean(mnb_log)
mnb_sd = np.std(mnb_log)
print( '\n')
print("Data split for multinomial NB train and test {} times.".format(N))
print( '\n')
print("Accuracy mean and S.D  are {:.3f} and {:.3f} ".format(mnb_mean, mnb_sd))
print( '\n')
print('Execution time in seconds: ' + str(round(execution_time_mnb,1)))

### Complement Naive Bayes (CNB)

CNB is a variation of the MNB model that is particularly suited for imbalanced data sets, see the [Sklearn ref ](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.ComplementNB.html#sklearn.naive_bayes.ComplementNB)

In [ ]:
start_time = time.time() # Record time to measure model runtime

N = 40 # Number of cycles
cnb_log = [] # log accuracy of N cycles

for i in range(N):
    # Split data into train and test sets
    X_train, X_test,y_train, y_test = model_selection.train_test_split(X,y,test_size = 0.25)

    # Feature extraction
    cv = feature_extraction.text.CountVectorizer()
    features = cv.fit_transform(X_train)

    # Fit model
    model_cnb = naive_bayes.ComplementNB()
    model_cnb.fit(features, y_train)
    
    # Test model
    features_test = cv.transform(X_test)
    cnb_log.append(model_cnb.score(features_test,y_test))

# Log CNB model runtime
execution_time_cnb = (time.time() - start_time)
    
cnb_mean = np.mean(cnb_log)
cnb_sd = np.std(cnb_log)

print( '\n')
print("Data split for complement NB train and test {} times.".format(N))
print( '\n')
print("Accuracy mean and S.D  are {:.3f} and {:.3f} ".format(cnb_mean, cnb_sd))
print( '\n')
print('CNB execution time in seconds: ' + str(round(execution_time_cnb,1)))

## Random Forest

The [RF classifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html#sklearn.ensemble.RandomForestClassifier) is another supervised ML model suited to text classification based on word frequency occurence in text.

In [ ]:
start_time = time.time() # Record time to measure model runtime

N = 40 # Number cycles
rf_log = [] # Log accuracy N cycles


for i in range(N):
    # As before, split data into train and test sets
    X_train, X_test,y_train, y_test = model_selection.train_test_split(X,y,test_size = 0.25)

    # Feature extraction
    cv = feature_extraction.text.CountVectorizer()
    features = cv.fit_transform(X_train)

    # create and train Random Forest Model
    forest_model = ensemble.RandomForestClassifier(random_state=1)
    forest_model.fit(features, y_train) # Sample weight ???

    # Test model
    features_test = cv.transform(X_test)
    rf_log.append(forest_model.score(features_test,y_test))

# Log RFmodel runtime
execution_time_rf = (time.time() - start_time)

rf_mean = np.mean(rf_log)
rf_sd = np.std(rf_log)
print("\n")
print("Data split for Random Forest train and test {} times.".format(N))
print("\n")
print("Accuracy mean and S.D  are {:.3f} and {:.3f} ".format(rf_mean, rf_sd))
print( '\n')
print('RF execution time in seconds: ' + str(round(execution_time_rf,1)))

## Compare model performance - accuracy and runtime

Accuracy mean and SD for SVM, MNB, CNB and RF collected into table and plotted.

In [ ]:
accuracy_data = {'Model': ['Support Vector Machine', 'Multinomial Naive Bayes', 'Complement Naive Bayes', 'Random Forest'], 'Mean Accuracy': [svm_mean, mnb_mean, cnb_mean, rf_mean], 'SD': [svm_sd, mnb_sd, cnb_sd, rf_sd],'Runtime (s)': [execution_time_svm/N, execution_time_mnb/N, execution_time_cnb/N, execution_time_rf/N]}

accuracy_df = pd.DataFrame(data=accuracy_data)
print("\n")
print("Comparison of the accuracy mean and SD for 100 cycles split-train-test")
print("\n")
accuracy_df.round(3)

# (+) Compare model performance with another dataset


# (+) Process text data to extract numerical data, e.g.

* Email length
* Link
* White space
* Punctuation
* Word count
* Word length
* Attachment

## References

* Log rocket [tutorial on SVM for spam detection](https://blog.logrocket.com/email-spam-detector-python-machine-learning/)
* Add ref for model selection
* Add ref word clouds